# Prerequisites
Install Theano and Lasagne using the following commands:

```bash
pip install -r https://raw.githubusercontent.com/Lasagne/Lasagne/master/requirements.txt
pip install https://github.com/Lasagne/Lasagne/archive/master.zip
```

Working in a virtual environment is recommended.

# Data preparation

Current code allows to generate geodesic patches from a collection of shapes represented as triangular meshes.
To get started with the pre-processing:
```
git clone https://github.com/jonathanmasci/ShapeNet_data_preparation_toolbox.git
```

The usual processing pipeline is show in ```run_forrest_run.m```. 
We will soon update this preparation stage, so perhaps better to start with our pre-computed dataset, and stay tuned! :-)

## Prepared data

All it is required to train on the FAUST_registration dataset for this demo is available for download at
https://www.dropbox.com/s/aamd98nynkvbcop/EG16_tutorial.tar.bz2?dl=0

# ICNN Toolbox

```bash
git clone https://github.com/sosiris/TAU16_seminar.git
```

![](http://www.people.usi.ch/mascij/EG16_tutorial/shapenet_architecture.png)

In [34]:
import sys
import os
import numpy as np
import scipy.io
import time

import theano
import theano.tensor as T
import theano.sparse as Tsp

import lasagne as L
import lasagne.layers as LL
import lasagne.objectives as LO
import lasagne.nonlinearities as LN
from lasagne.layers.normalization import batch_norm

sys.path.append('..')
from icnn import utils_lasagne, dataset, snapshotter

## Data loading

In [35]:
reload(dataset)
base_path = './dataset/FAUST_registrations/data/diam=200/'

ds = dataset.ClassificationDatasetPatchesMinimal(
    'FAUST_registrations_train.txt', 'FAUST_registrations_test.txt',
    os.path.join(base_path, 'descs', 'shot'),
    os.path.join(base_path, 'patch_aniso', 'alpha=100_nangles=016_ntvals=005_tmin=6.000_tmax=24.000_thresh=99.900_norm=L1'), 
    None, 
    os.path.join(base_path, 'lbo'),
    os.path.join(base_path, 'labels'),
    epoch_size=50)

Loading train descs
elapsed time 8.207000
Loading test descs
elapsed time 1.846000
Loading train patches
elapsed time 32.311000
Loading test patches
elapsed time 7.828000
Loading train LB bases
elapsed time 4.954000
Loading test LB bases
elapsed time 1.168000
Loading train labels
elapsed time 0.477000
Loading test labels
elapsed time 0.162000


## Network definition

In [36]:
reload(utils_lasagne)
nin = 544
nclasses = 6890
l2_weight = 1e-5
c_weight = 1e-2
ref_lbo = T.constant(ds.train_lbo[0])

def get_model(inp, patch_op, lb_op):
    print(LL.get_output_shape(inp))
    icnn = batch_norm(utils_lasagne.GCNNLayer([inp, patch_op], 16, nrings=5, nrays=16))
    print(LL.get_output_shape(icnn))
    icnn = batch_norm(utils_lasagne.GCNNLayer([icnn, patch_op], 32, nrings=5, nrays=16))
    print(LL.get_output_shape(icnn))
    icnn = batch_norm(utils_lasagne.GCNNLayer([icnn, patch_op], 64, nrings=5, nrays=16))
    print(LL.get_output_shape(icnn))
    # TODO: add functional mapping computation and application (maybe instead of the dense layer?)
    fmap = utils_lasagne.FMAPLayer([icnn, lb_op], neigen=30,
                                 nonlinearity=LN.rectify)
    print(LL.get_output_shape(fmap))
    ffn = LL.DenseLayer(fmap, nclasses, nonlinearity=utils_lasagne.log_softmax)

    print(LL.get_output_shape(ffn))
    return ffn, fmap.C

inp = LL.InputLayer(shape=(None, nin))
patch_op = LL.InputLayer(input_var=Tsp.csc_fmatrix('patch_op'), shape=(None, None))
lb_op = LL.InputLayer(input_var=T.matrix('lb_op'), shape=(None, None))

ffn, C = get_model(inp, patch_op, lb_op)

# L.layers.get_output -> theano variable representing network
output = LL.get_output(ffn)
print(LL.get_output_shape(ffn))
pred = LL.get_output(ffn, deterministic=True)  # in case we use dropout

# target theano variable indicatind the index a vertex should be mapped to wrt the latent space
target = T.ivector('idxs')

# to work with logit predictions, better behaved numerically
cla = utils_lasagne.categorical_crossentropy_logdomain(output, target, nclasses).mean()
acc = LO.categorical_accuracy(pred, target).mean()

# a bit of regularization is commonly used
regL2 = L.regularization.regularize_network_params(ffn, L.regularization.l2)

# regularize fanuctional map to penalize non diagonal maps
regC = L.regularization.apply_penalty(C, utils_lasagne.diag_penalty)

cost = cla + l2_weight * regL2 + c_weight * regC

(None, 544)
(None, 16)
(None, 32)
(None, 64)
(None, 64)
(None, 6890)
(None, 6890)


## Define the update rule, how to train

In [37]:
params = LL.get_all_params(ffn, trainable=True)
grads = T.grad(cost, params)
# computes the L2 norm of the gradient to better inspect training
grads_norm = T.nlinalg.norm(T.concatenate([g.flatten() for g in grads]), 2)

# Adam turned out to be a very good choice for correspondence
updates = L.updates.adam(grads, params, learning_rate=0.001)

## Compile

In [38]:
funcs = dict()
funcs['train'] = theano.function([inp.input_var, patch_op.input_var, lb_op.input_var, target],
                                 [cost, cla, l2_weight * regL2, grads_norm, acc], updates=updates,
                                 on_unused_input='warn')
funcs['acc_loss'] = theano.function([inp.input_var, patch_op.input_var, lb_op.input_var, target],
                                    [acc, cost], on_unused_input='warn')
funcs['predict'] = theano.function([inp.input_var, patch_op.input_var, lb_op.input_var],
                                   [pred], on_unused_input='warn')

# Training (a bit simplified)

In [ ]:
n_epochs = 50
eval_freq = 1

start_time = time.time()
best_trn = 1e5
best_tst = 1e5

kvs = snapshotter.Snapshotter('demo_training.snap')

for it_count in xrange(n_epochs):
    tic = time.time()
    b_l, b_c, b_s, b_r, b_g, b_a = [], [], [], [], [], []
    for x_ in ds.train_iter():
        tmp = funcs['train'](*x_)

        # do some book keeping (store stuff for training curves etc)
        b_l.append(tmp[0])
        b_c.append(tmp[1])
        b_r.append(tmp[2])
        b_g.append(tmp[3])
        b_a.append(tmp[4])
    epoch_cost = np.asarray([np.mean(b_l), np.mean(b_c), np.mean(b_r), np.mean(b_g), np.mean(b_a)])
    print(('[Epoch %03i][trn] cost %9.6f (cla %6.4f, reg %6.4f), |grad| = %.06f, acc = %7.5f %% (%.2fsec)') %
                 (it_count, epoch_cost[0], epoch_cost[1], epoch_cost[2], epoch_cost[3], epoch_cost[4] * 100, 
                  time.time() - tic))

    if np.isnan(epoch_cost[0]):
        print("NaN in the loss function...let's stop here")
        break

    if (it_count % eval_freq) == 0:
        v_c, v_a = [], []
        for x_ in ds.test_iter():
            tmp = funcs['acc_loss'](*x_)
            v_a.append(tmp[0])
            v_c.append(tmp[1])
        test_cost = [np.mean(v_c), np.mean(v_a)]
        print(('           [tst] cost %9.6f, acc = %7.5f %%') % (test_cost[0], test_cost[1] * 100))

        if epoch_cost[0] < best_trn:
            kvs.store('best_train_params', [it_count, LL.get_all_param_values(ffn)])
            best_trn = epoch_cost[0]
        if test_cost[0] < best_tst:
            kvs.store('best_test_params', [it_count, LL.get_all_param_values(ffn)])
            best_tst = test_cost[0]
print("...done training %f" % (time.time() - start_time))

[Epoch 000][trn] cost  8.319470 (cla 8.3040, reg 0.0060), |grad| = 0.137901, acc = 0.11001 % (319.09sec)
           [tst] cost  7.825659, acc = 0.19376 %


ERROR:root:[KVP] Overwriting group best_train_params!
ERROR:root:[KVP] Overwriting group best_test_params!


[Epoch 001][trn] cost  7.480322 (cla 7.4541, reg 0.0167), |grad| = 0.274470, acc = 0.30392 % (261.87sec)
           [tst] cost  7.205690, acc = 0.54354 %


ERROR:root:[KVP] Overwriting group best_train_params!
ERROR:root:[KVP] Overwriting group best_test_params!


# Test phase
Now that the model is train it is enough to take the fwd function and apply it to new data.

In [23]:
rewrite = True

out_path = './dumps/' 
kvs.load('best_test_params')

print "Saving output to: %s" % out_path

if not os.path.isdir(out_path) or rewrite==True:
    try:
        os.makedirs(out_path)
    except:
        pass
    
    a = []
    for i,d in enumerate(ds.test_iter()):
        fname = os.path.join(out_path, "%s" % ds.test_fnames[i])
        print fname,
        tmp = funcs['predict'](d[0], d[1], d[2])[0]
        a.append(np.mean(np.argmax(tmp, axis=1).flatten() == d[3].flatten()))
        scipy.io.savemat(fname, {'desc': tmp})
        print ", Acc: %7.5f %%" % (a[-1] * 100.0)
    print "\nAverage accuracy across all shapes: %7.5f %%" % (np.mean(a) * 100.0)
else:
    print "Model predictions already produced."

Saving output to: ./dumps/
./dumps/tr_reg_080.mat

TypeError: Missing required input: lb_op

# Results

![](http://www.people.usi.ch/mascij/EG16_tutorial/shapenet_corr.png)